In [19]:
# from transformers import BertConfig, BertForMaskedLM, BertTokenizer
# from transformers import PreTrainedTokenizer
# from transformers import get_linear_schedule_with_warmup
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torch.utils.data import random_split
# from torch.nn.functional import pad
# from datasets import load_dataset
from tqdm.auto import tqdm
# from itertools import cycle
# import wandb


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

In [20]:
import copy
from collections import defaultdict

# import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# from   matplotlib import colors
# import seaborn as sns

import json
import os
from pathlib import Path
from glob import glob
import gc
import pickle
import sys

# from subprocess import Popen, PIPE, STDOUT

# re-arc data

In [21]:
sys.path.append('re-arc')
from main import get_generators


all_generators_map = get_generators()

In [22]:
def generate(n_samples=1_000_000, n_train_boards_per_sample=3, ub=1.0):
    n_tasks = len(all_generators_map)
    assert n_samples % n_tasks == 0
    n_samples_per_task = n_samples // n_tasks

    # max_board_numel = 100

    # def is_exceed_sample(sample):
    #     return (
    #         np.size(sample['input']) > max_board_numel
    #         or
    #         np.size(sample['output']) > max_board_numel
    #     )

    # def is_exceed_task(task):
    #     is_exceed_train = any(map(is_exceed_sample, task['train']))
    #     is_exceed_test = any(map(is_exceed_sample, task['test']))
    #     return is_exceed_train or is_exceed_test

    def convert_board_tolist(board):
        return np.asarray(board).tolist()

    def convert_task_sample_tolist(task_sample):
        task_sample['input'] = convert_board_tolist(task_sample['input'])
        task_sample['output'] = convert_board_tolist(task_sample['output'])
        return task_sample

    def convert_datasample_tolist(datasample):
        for i, train_task_sample in enumerate(datasample['train']):
            train_task_sample = convert_task_sample_tolist(train_task_sample)
            datasample['train'][i] = train_task_sample

        for i, test_task_sample in enumerate(datasample['test']):
            test_task_sample = convert_task_sample_tolist(test_task_sample)
            datasample['test'][i] = test_task_sample

        return datasample

    data = dict()

    all_keys = list(all_generators_map.keys())

    for i, task_key in enumerate(tqdm(all_keys, desc='tasks'), start=1):
        task_generator = all_generators_map[task_key]

        samples = []

        # pbar = tqdm(total=n_samples_per_task, desc='task %d/%d' % (i, n_tasks), leave=False)
        while True:
            sampled = []
            while True:
                try:
                    one_sample = task_generator(0, ub)
                except IndexError:
                    continue
                else:
                    sampled.append(one_sample)

                if len(sampled) >= n_train_boards_per_sample + 1:
                    break

            train_samples = sampled[:n_train_boards_per_sample]
            test_samples = sampled[-1:]
            sample = {
                'train': train_samples,
                'test': test_samples,
            }

            sample = convert_datasample_tolist(sample)

            # if is_exceed_task(sample):
            #     continue

            # pbar.update(1)
            samples.append(sample)

            if len(samples) >= n_samples_per_task:
                break
                # pbar.close()

        data[task_key] = samples

    return data

In [ ]:
data = generate(ub=0.05)

In [6]:
# with open('data/rearc1M.json', 'w') as f:
#     json.dump(data, f, indent=2)

In [7]:
with open('data/rearc1M.pkl', 'wb') as f:   
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
! ls data -l

In [14]:
import pickle

with open('data/rearc1M.pkl', 'rb') as f:
    other_data = pickle.load(f)

In [ ]:
len(other_data)

In [8]:
# next(iter(data['007bbfb7'][0].values()))['train'][0]['input']

In [9]:
# i = 392
# all_keys = list(all_generators_map.keys())
# g = all_generators_map[all_keys[i]]
# np.size(g(0, 0.05)['input']), np.size(g(0, 0.1)['output'])